In [1]:
import simtk.openmm.app
import simtk.openmm
import simtk.unit as unit
import configparser
import numpy as np
import itertools
import scipy.spatial.distance as sdist
import os
import pdbfixer
import pandas
import subprocess
import nose

In [2]:
import openna

The location is /mnt/f/linux/script/openna
4.184


In [3]:
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(openna.xml)
s=forcefield.createSystem(top)

In [4]:
rna=openna.RNA.fromCoarsePDB('clean.pdb')
rna.periodic=False
rna.parseConfigurationFile(configuration_file=f'./openna.conf')

In [5]:
print(rna.pair_definition)

  nucleic Base1 Base2  sigma  alpha  rang  epsilon  torsion      t1      t2
0     RNA     A     U  5.861      2    12  3.99874    50.17  160.91  140.49
1     RNA     U     A  5.861      2    12  3.99874    50.17  160.91  140.49
2     RNA     G     C  5.528      2    12  5.06241    38.33  165.25  147.11
3     RNA     C     G  5.528      2    12  5.06241    38.33  165.25  147.11


In [6]:
import forceTerm
all_forces = dict(Backbone=forceTerm.Backbone,
                 Angle=forceTerm.Angle,
                 Dihedral=forceTerm.Dihedral,
                 BaseStacking=forceTerm.BaseStacking,
                 BasePair=forceTerm.BasePair,
                 Exclusion=forceTerm.Exclusion,
                 Electrostatics=forceTerm.Electrostatics)

for each_one in all_forces:
    if each_one == 'Electrostatics':
        force_now = all_forces[each_one](rna)
        s.addForce(force_now)
    elif each_one == 'BasePair':
        force_now = all_forces[each_one](rna)
        force_now.addForce(s)
    else:
        force_now = all_forces[each_one](rna)
        s.addForce(force_now)
    

A C
[0]
nucleic        RNA
Base1            C
Base2            G
sigma        5.528
alpha            2
rang            12
epsilon    5.06241
torsion      38.33
t1          165.25
t2          147.11
Name: 3, dtype: object
A C
[2]
nucleic        RNA
Base1            C
Base2            G
sigma        5.528
alpha            2
rang            12
epsilon    5.06241
torsion      38.33
t1          165.25
t2          147.11
Name: 3, dtype: object
A C
[5]
nucleic        RNA
Base1            C
Base2            G
sigma        5.528
alpha            2
rang            12
epsilon    5.06241
torsion      38.33
t1          165.25
t2          147.11
Name: 3, dtype: object


In [7]:
print(rna.pair_definition)

  nucleic Base1 Base2  sigma  alpha  rang  epsilon  torsion      t1      t2
0     RNA     A     U  5.861      2    12  3.99874    50.17  160.91  140.49
1     RNA     U     A  5.861      2    12  3.99874    50.17  160.91  140.49
2     RNA     G     C  5.528      2    12  5.06241    38.33  165.25  147.11
3     RNA     C     G  5.528      2    12  5.06241    38.33  165.25  147.11


In [8]:
import numpy as np
temperature=300 * simtk.openmm.unit.kelvin
#platform_name='CUDA'

platform_name='Reference'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top, s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

-37.11984582103105


In [9]:
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

TotalEnergy -37.119846 kJ/mol


In [10]:
energies = {}
for force_name, force in all_forces.items():
    force = force(rna)
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in all_forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

A C
[0]
nucleic        RNA
Base1            C
Base2            G
sigma        5.528
alpha            2
rang            12
epsilon    5.06241
torsion      38.33
t1          165.25
t2          147.11
Name: 3, dtype: object
A C
[2]
nucleic        RNA
Base1            C
Base2            G
sigma        5.528
alpha            2
rang            12
epsilon    5.06241
torsion      38.33
t1          165.25
t2          147.11
Name: 3, dtype: object
A C
[5]
nucleic        RNA
Base1            C
Base2            G
sigma        5.528
alpha            2
rang            12
epsilon    5.06241
torsion      38.33
t1          165.25
t2          147.11
Name: 3, dtype: object
Backbone 0.0 kJ/mol
Angle 19.900579 kJ/mol
Dihedral -59.961694 kJ/mol
BaseStacking -59.961694 kJ/mol
BasePair 0.0 kJ/mol
Exclusion 0.0 kJ/mol
Electrostatics 2.94127 kJ/mol


In [11]:
import sys
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
pdb_reporter=simtk.openmm.app.PDBReporter(f'output.pdb', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(energy_reporter)
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(50000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,-12.716939378635422,273.6438466827353
2000,3.999999999999781,2.0003211983117732,305.6486815081939
3000,5.999999999999561,-8.54443958158165,237.65931191665604
4000,7.999999999999341,-25.149202025844705,267.5962319585667
5000,10.000000000000009,-10.556630707945176,364.91906668033323
6000,12.000000000000677,3.598204390822898,305.51866019087277
7000,14.000000000001345,-17.023492812895874,318.2979236111423
8000,16.00000000000201,13.676536860126596,310.1471837275559
9000,18.000000000000902,18.646169817524694,349.754535078527
10000,19.999999999999794,33.79791734809185,394.4388243467503
11000,21.999999999998685,40.52510580547836,411.84278319122654
12000,23.999999999997577,59.32995805661276,305.2476229698012
13000,25.99999999999647,16.978053700204086,281.039895878374
14000,27.99999999999536,55.1234491943869,316.8908499386752
15000,29.99999999999425,58.553347929891345,297.3129142773247
16000,31.99999999999